# Example Notebook to showcasing how we interact with JTReaders

In [1]:
# First change dir to JTR parent
import os
os.chdir('..')

### Bookkeeping of all existing readers: `readers.py`

In [2]:
import jtr.readers as readers

In [3]:
print("Existing models:\n%s" % ", ".join(readers.readers.keys()))

Existing models:
example_reader, modelf_reader, distmult_reader, complex_reader, transe_reader, fastqa_reader, cbow_xqa_reader, snli_reader


### Create a reader

In [ ]:
%%script bash
bash data/GloVe/download.sh

glove.6B.50d.txt already exists! Doing nothing!


In [ ]:
from jtr.io.embeddings.embeddings import load_embeddings
from jtr.util.vocab import Vocab

# we need a vocabulary (with embeddings for our fastqa_reader, but this is not always necessary)
embeddings = load_embeddings('data/GloVe/glove.6B.50d.txt', 'glove')
vocab = Vocab(emb=embeddings, init_from_embeddings=True)

# and a config
config = {"batch_size": 1, "repr_dim": 10, "repr_dim_input": embeddings.lookup.shape[1]}

In [ ]:
# create example reader
from jtr.core import SharedVocabAndConfig

svac = SharedVocabAndConfig(vocab, config)
fastqa_reader = readers.readers["fastqa_reader"](svac)

### Setting up a reader from training data 

In [ ]:
from jtr.data_structures import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet.jtr.json')
# all parameters are initialized after this call
fastqa_reader.setup_from_data(train_data)

### Saving the reader

In [ ]:
fastqa_reader.store("/tmp/fastqa_reader")

In [ ]:
%%sh
ls /tmp/fastqa_reader/

### Loading the reader

In [ ]:
# we can simply load a setup reader
fastqa_reader.load("/tmp/fastqa_reader")

In [ ]:
# or setup a new reader from file 
import tensorflow as tf
from jtr.util.vocab import Vocab

# reset graph -> computation graph is gone
tf.reset_default_graph()
svac = SharedVocabAndConfig(None, None)
fastqa_reader = readers.readers["fastqa_reader"](svac)
fastqa_reader.setup_from_file("/tmp/fastqa_reader")

### Applying the reader

In [ ]:
from jtr.data_structures import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet.jtr.json')

# take a list of inputs, e.g., from our training data
questions = [q for q, a in train_data]

In [ ]:
for a in fastqa_reader(questions):
    print("{}, {}, {}".format(a.score, a.text, a.span))